In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../../..'))

import importlib

import contextlib

import json
import os
import random

import numpy as np
import torch
from collections import defaultdict
from utils import get_default_path

import time
from utils import Stopwatch
import datetime

import ai.stabledisco as sd
import ai.torchmodules as torchmodules
import ai.torchmodules.data as torchdata
import ai.torchmodules.utils as torchutils
import ai.stabledisco.utils as sdutils
import ai.stabledisco.data as sddata
import clip
import torch
import torch.nn as nn
import pandas as pd

In [2]:
df_file_path = get_default_path("large_datasets", "laion_features.feather")
if "prompt_dataframe" in dir():
    del prompt_dataframe

print("Loading rating")
text_rating = pd.read_feather(df_file_path, columns=['aesthetic'])['aesthetic']
text_rating = np.array(text_rating).astype(np.float32)
text_rating = torch.tensor(text_rating)

Loading rating


In [ ]:
from datasets import load_dataset

dataset = load_dataset("laion/laion2B-en-aesthetic", cache_dir=get_default_path("large_datasets/laion_full"))

In [3]:
dataset = dataset.remove_columns(["URL", "WIDTH", "HEIGHT", "similarity", "hash", "punsafe", "pwatermark"])

In [6]:
dataset = dataset["train"]

In [7]:
aes_scores = np.array(dataset["aesthetic"])

In [4]:
print(torch.min(text_rating))
print(torch.max(text_rating))
print(torch.mean(text_rating))
print(torch.std(text_rating))

tensor(7.0000)
tensor(10.4578)
tensor(7.3842)
tensor(0.3389)


In [9]:
dataset = dataset.remove_columns(["URL", "WIDTH", "HEIGHT", "similarity", "hash", "punsafe", "pwatermark"])
def is_valid_sd_training_row(row, dimension_threshold=512, aesthetic_threshold=5):
    if pd.isnull(row['WIDTH']) or row['WIDTH'] < dimension_threshold:
        return False
    if pd.isnull(row['HEIGHT']) or row['HEIGHT'] < dimension_threshold:
        return False
    
    return True
    
dataset = dataset.filter(is_valid_sd_training_row)

ValueError: Column name URL not in the dataset. Current columns in the dataset: ['TEXT', 'aesthetic']

In [73]:
df_dict = dataset.to_dict()
del dataset
torchutils.torch_garbage_collect()

laion_df = pd.DataFrame(df_dict)
del df_dict
laion_df.drop_duplicates(subset="prompt", ignore_index=True, inplace=True)

In [3]:
df_path = get_default_path("large_datasets", "laion.feather")

In [ ]:
torchutils.torch_garbage_collect()
laion_df.to_feather(df_path)

In [4]:
laion_df = pd.read_feather(df_path)

In [5]:
df_file_path = get_default_path("large_datasets", "aug_prompts.feather")
if "prompt_dataframe" in dir():
    del prompt_dataframe
prompt_dataframe = pd.read_feather(df_file_path)

In [6]:
prompt_dataframe = pd.concat([prompt_dataframe,  laion_df], ignore_index=True)
print(len(prompt_dataframe))
prompt_dataframe.drop_duplicates(subset="prompt", ignore_index=True, inplace=True)
prompt_dataframe.reset_index(drop=True, inplace=True)
prompt_dataframe = prompt_dataframe.sample(frac=1).reset_index(drop=True)
len(prompt_dataframe) 

19476387


16728531

In [7]:
del laion_df
prompt_dataframe.to_feather(get_default_path("large_datasets", "aug_prompts.feather"))

In [35]:
for row in laion_df.itertuples():
    lower_prompt = row.prompt.lower()
    if "mystery" in lower_prompt and "guitar" in lower_prompt:
        print(lower_prompt)
    if "joe" in lower_prompt and "penna" in lower_prompt:
        print(lower_prompt)

joewyle happy birthday party supplies banner for baby boys birthday party decoration, polka dot print pennant flag banner blue
joe dimaggio charlie keller tommy henrich nyy going into the 1941 ws a team that had won the pennant by new york yankees baseball american league mlb uniforms
with their 100th victory of the year, theyankeesclinch theamerican leaguepennant asgeorge pipgrasbeats theindiansatcleveland stadium, 8 – 3. yankee managerjoe mccarthy, who captured a flag with the1929 cubs, becomes the first manager to win pennants in both the american andnational leagues.
joewyle happy birthday party supplies banner for baby boys birthday party decoration, polka dot print pennant flag banner blue
joewyle happy birthday party supplies banner for baby boys birthday party decoration, polka dot print pennant flag banner blue


In [24]:
print('\n'.join(list(prompt_dataframe.loc[:1000, "prompt"])))

10 things we learned about fallout 76
tropical twitter background free Free twitter wallpaper
Ravelry: JulieAnny's Stained Glass Afghan Square pattern by Julie Yeager
Pencil and watercolor sketch of Maitreyi Ramakrishnan looking effortlessly glamorous in a windbreaker and gold jewelry.
Classic Tractor Motorcycle Legoingly Technic Creator Model Building Blocks Bricks Technician Toys For Children Christmas Gift
still closeup photo of cat sitting on the moon watching planet earth. 2 4 0 mm, on the moon, cat, photorealistic, photography, night directed by wes anderson
Wedding glamping
grainy film photo of a man with his mouth wide open, and showin multiple rows of teeth, creepy, unsettling 
"""Kitten Love"" original fine art by Karen Margulis"
Make Your Own Almond Milk
zeus god in ancient sci-fi armor, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, ArtStation, art by giger and greg rutkowski and Hikari Shimoda and Edmund Blair Leighton and Ch

In [31]:
from collections import defaultdict
from open_clip.tokenizer import _tokenizer as clip_tokenizer
sot_token = clip_tokenizer.encoder["<start_of_text>"]
eot_token = clip_tokenizer.encoder["<end_of_text>"]

token_cnt = defaultdict(int)
for idx in range(sot_token):
    if idx not in token_cnt:
        token_cnt[idx] = 0

for row in laion_df.itertuples():
    for token in row.text_tokens[1:]:
        if token == eot_token:
            break
        token_cnt[token] += 1
        
    if row[0] >= 9*len(laion_df)//10:
        break

In [32]:
cnt_token_pairs = [(val, key) for key, val in token_cnt.items()]
cnt_token_pairs.sort()

In [33]:
cnts = [x[0] for x in cnt_token_pairs]

In [ ]:
for perc in range(1,100):
    print(f"Perc {perc}: {np.percentile(cnts, perc)}")

In [5]:
laion_df.rename(columns={"TEXT": "prompt"}, inplace=True)

In [ ]:
laion_df["text_tokens"] = clip.tokenize(laion_df["prompt"])

In [27]:
tokens = clip.tokenize(laion_df.loc[start:start+step_size, "prompt"], truncate=True)
selected =  laion_df.loc[start:start+step_size, "text_tokens"]

In [9]:
laion_df.sort_values('prompt',inplace=True, ascending=False)

In [13]:
laion_df.reset_index(drop=True, inplace=True)

In [ ]:
def calc_jaccard_similarity(text_a, text_b):
    doc1_tokens=set(text_a.lower().split())
    doc2_tokens=set(text_b.lower().split())
    div_inter = len(doc1_tokens.union(doc2_tokens))
    if div_inter == 0:
        return 0
    return len(doc1_tokens.intersection(doc2_tokens))/div_inter



last_text = ""
threshold = 0.80
to_remove = []
next_remove_candidates = []
max_similar = 2
for cnt, row in enumerate(laion_df.itertuples(True)):
    if cnt % 100000 == 0:
        print(f"Removed {len(to_remove)} at cnt {cnt}")

    if len(row.prompt) == 0:
        print(row.prompt)
        to_remove.append(row[0])
    jaccard_similarity = calc_jaccard_similarity(last_text, row.prompt)
    if jaccard_similarity > threshold:
        next_remove_candidates.append((f"{row.prompt}", row[0]))
    else:
        next_remove_candidates.sort(key=lambda x: len(x))
        next_remove_candidates = [x[1] for x in next_remove_candidates]
        if len(next_remove_candidates) > max_similar:
            to_remove += next_remove_candidates[:-(max_similar)]
        next_remove_candidates = []
        last_text = row.prompt
        
print(len(to_remove))

In [18]:
print(len(laion_df))
laion_df.drop(laion_df.index[to_remove], inplace=True)
print(len(laion_df))
laion_df.reset_index(drop=True, inplace=True)
laion_df = laion_df.sample(frac=1).reset_index(drop=True)

14669680
13279821


In [3]:
df_path = get_default_path("large_datasets", "laion.feather")
laion_df = pd.read_feather(df_path)
print(len(laion_df))
sample_df = laion_df.sample(n=2000000).reset_index(drop=True)

df_path = get_default_path("large_datasets", "aug_prompts.feather")
sample_df.to_feather(df_path)

13279821


In [ ]:
import markovify
from markovify import Text

sample_markov = markovify.NewlineText(laion_df["prompt"])

In [78]:
step_size = 100000
all_tokens = []
for start in range(0, len(laion_df), step_size):
    print(f"{start} of {len(laion_df)} | {start/len(laion_df):0.2f}% finished")
    all_tokens += clip.tokenize(laion_df.loc[start:start+step_size-1, "prompt"], truncate=True).numpy().tolist()

laion_df["text_tokens"] = all_tokens

0 of 13279821 | 0.00% finished
100000 of 13279821 | 0.01% finished
200000 of 13279821 | 0.02% finished
300000 of 13279821 | 0.02% finished
400000 of 13279821 | 0.03% finished
500000 of 13279821 | 0.04% finished
600000 of 13279821 | 0.05% finished
700000 of 13279821 | 0.05% finished
800000 of 13279821 | 0.06% finished
900000 of 13279821 | 0.07% finished
1000000 of 13279821 | 0.08% finished
1100000 of 13279821 | 0.08% finished
1200000 of 13279821 | 0.09% finished
1300000 of 13279821 | 0.10% finished
1400000 of 13279821 | 0.11% finished
1500000 of 13279821 | 0.11% finished
1600000 of 13279821 | 0.12% finished
1700000 of 13279821 | 0.13% finished
1800000 of 13279821 | 0.14% finished
1900000 of 13279821 | 0.14% finished
2000000 of 13279821 | 0.15% finished
2100000 of 13279821 | 0.16% finished
2200000 of 13279821 | 0.17% finished
2300000 of 13279821 | 0.17% finished
2400000 of 13279821 | 0.18% finished
2500000 of 13279821 | 0.19% finished
2600000 of 13279821 | 0.20% finished
2700000 of 13279

In [82]:
laion_df.to_feather(df_path)

In [4]:
clip_model = sdutils.load_clip_model()

In [7]:
tokens = laion_df["text_tokens"].to_numpy()

In [17]:
tokens = np.vstack(tokens).astype(np.int64)

In [18]:
tokens = torch.from_numpy(tokens)

In [21]:
torchutils.torch_garbage_collect()

In [22]:
text_features = []

step_size = 10000
for start in range(0, len(tokens), step_size):
    if start % (step_size*10) == 0:
        print(f"Finished {start} or {len(tokens)}")
    text_features += clip_model.features_from_tokens(tokens[start:start+step_size].cuda()).cpu().numpy().tolist()


Finished 0 or 13279821
Finished 100000 or 13279821
Finished 200000 or 13279821
Finished 300000 or 13279821
Finished 400000 or 13279821
Finished 500000 or 13279821
Finished 600000 or 13279821
Finished 700000 or 13279821
Finished 800000 or 13279821
Finished 900000 or 13279821



KeyboardInterrupt



In [ ]:
dataset["text_features"] = text_features
del text_features

In [ ]:
laion_df.to_feather(df_path)